<a href="https://colab.research.google.com/github/kavyajeetbora/EAST/blob/master/EAST_training_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Imports

[**Tasks - Incidental Scene Text**](https://rrc.cvc.uab.es/?ch=4&com=tasks)

In [0]:
import os
import time
from PIL import Image
from torch.utils.model_zoo import load_url
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
import shutil
from google.colab import files

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [12]:
%%time
!rm -r /content/cloned-repo
%cd ..
!git clone -l -s git://github.com/kavyajeetbora/EAST.git /content/cloned-repo
%cd /content/cloned-repo
!ls

rm: cannot remove '/content/cloned-repo': No such file or directory
/
Cloning into '/content/cloned-repo'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (84/84), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 166 (delta 46), reused 0 (delta 0), pack-reused 82
Receiving objects: 100% (166/166), 6.91 MiB | 10.17 MiB/s, done.
Resolving deltas: 100% (91/91), done.
/content/cloned-repo
dataset.py		    EAST_resnet.py		    loss.py
detect.py		    EAST_training.ipynb		    model.py
dsiplay_ground_truth.ipynb  EAST_training_vgg16_bn_2.ipynb  model_resnet.py
EAST_detection.ipynb	    EAST_training_vgg16_bn.ipynb    train.py
CPU times: user 95.2 ms, sys: 90.3 ms, total: 186 ms
Wall time: 8.39 s


In [0]:
from dataset import *

In [21]:
%%time

# AI4Bharat dataset
image_folder = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Images/Images.Zip (Unzipped Files)'
label_folder = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/AI4Bharat Dataset/Training Set/Annotations/Annotations.Zip (Unzipped Files)'

training_dataset = custom_dataset(image_folder, label_folder)
print(len(training_dataset))

5000
CPU times: user 19.1 ms, sys: 3.46 ms, total: 22.6 ms
Wall time: 80.4 ms


## Loss for one batch

In [23]:
%%time
train_loader = DataLoader(training_dataset, shuffle = True, batch_size=3, drop_last=True)
imgs, gt_score, gt_geo, ignored_maps = next(iter(train_loader))
print(imgs.size(), gt_score.size(), gt_geo.size(), ignored_maps.size())
print(torch.sum(gt_score))

torch.Size([3, 3, 512, 512]) torch.Size([3, 1, 128, 128]) torch.Size([3, 5, 128, 128]) torch.Size([3, 1, 128, 128])
tensor(338.)
CPU times: user 759 ms, sys: 1.51 s, total: 2.27 s
Wall time: 3.51 s


In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [25]:
%%time
from model import EAST
model = EAST()
pretrained_path = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/EAST scene text detector/east_vgg16.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
model = model.to(device)
imgs, gt_score, gt_geo, ignored_maps = imgs.to(device), gt_score.to(device), gt_geo.to(device), ignored_maps.to(device)
pred_score, pred_geo = model(imgs)

print(pred_score.size(), pred_geo.size())

Downloading: "http://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:16<00:00, 33.9MB/s]


torch.Size([3, 1, 128, 128]) torch.Size([3, 5, 128, 128])
CPU times: user 3.39 s, sys: 2.1 s, total: 5.49 s
Wall time: 21.5 s


In [26]:
from loss import Loss

loss_fn = Loss()
loss_fn(gt_score, pred_score, gt_geo, pred_geo, ignored_maps)

classify loss is 0.38455987, angle loss is 0.02235992, iou loss is 0.35559922


tensor(0.9638, device='cuda:0', grad_fn=<AddBackward0>)

## Training Full Batch 

fine tuning pretrained model [east_vgg16.pth](https://drive.google.com/file/d/1AFABkJgr5VtxWnmBU3XcfLJvpZkC2TAg/view)

In [0]:
model = EAST()
pretrained_path = '/content/drive/My Drive/Colab Notebooks (1)/padh.ai.notebooks/15. Object Detection/EAST scene text detector/east_7_11_19.pth'
state_dict = torch.load(pretrained_path)
model.load_state_dict(state_dict)
model = model.to(device)

In [0]:
def train(model, train_img_path, train_gt_path, batch_size, lr, epoch_iter, num_workers=4, interval=1):
    file_num = len(os.listdir(train_img_path))
    print(file_num)
    trainset = custom_dataset(train_img_path, train_gt_path)
    train_loader = data.DataLoader(trainset, batch_size=batch_size, \
                                   shuffle=True, num_workers=num_workers, drop_last=True)
	
    criterion = Loss()
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
    scheduler = lr_scheduler.StepLR(optimizer, step_size=epoch_iter//2, gamma=0.5)
    loss_curve = []
    
    for epoch in range(epoch_iter):
      model.train()
      scheduler.step()
      epoch_loss = 0
      epoch_time = time.time()
      for i, (img, gt_score, gt_geo, ignored_map) in enumerate(train_loader):
        start_time = time.time()
        img, gt_score, gt_geo, ignored_map = img.to(device), gt_score.to(device), gt_geo.to(device), ignored_map.to(device)
        pred_score, pred_geo = model(img)
        loss = criterion(gt_score, pred_score, gt_geo, pred_geo, ignored_map)

        epoch_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print('Epoch is [{}/{}], mini-batch is [{}/{}], time consumption is {:.8f}, batch_loss is {:.8f}'.format(\
                  epoch+1, epoch_iter, i+1, int(file_num/batch_size), time.time()-start_time, loss.item()))
        
      print('epoch_loss is {:.8f}, epoch_time is {:.8f}'.format(epoch_loss/int(file_num/batch_size), time.time()-epoch_time))
      print(time.asctime(time.localtime(time.time())))
      print('='*50)
      
      
      if (epoch + 1) % interval == 0:
        state_dict = model.state_dict()
        torch.save(state_dict, 'model_epoch_{}.pth'.format(epoch+1))
        try:
          files.download('model_epoch_{}.pth'.format(epoch+1))
          print('Load Successfully downloaded')
        except:
          print('Couldn\'t download the weights')
          pass

      ## plotting the loss curve
      loss_curve.append(epoch_loss/int(file_num/batch_size))

    return loss_curve

In [2]:
%%time
loss_curve = train(model, image_folder, label_folder, batch_size=3, lr=1e-5, epoch_iter=5, interval = 1)

NameError: ignored

In [0]:
plt.plot(loss_curve)

[Adam and AdamW algorithms](https://www.fast.ai/2018/07/02/adam-weight-decay/)